# Forecasting population in Vilnius districts by age group and gender

Jonas Vitkauskas | 
Paulina Udes |
Donatas Goštautas | 

### Uploading libraries

In [2]:
import pandas as pd
import numpy as np
from prophet import Prophet
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import Dropdown, VBox, HBox, Button, Output
from IPython.display import display
import logging
import warnings
import contextlib
from sklearn.linear_model import LinearRegression

### Getting rid of warning and logging messages

In [ ]:
# Suppress logging messages from cmdstanpy
logger = logging.getLogger('cmdstanpy')
logger.setLevel(logging.ERROR)
for handler in logger.handlers:
    handler.setLevel(logging.ERROR)

# Suppress SettingWithCopyWarning
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

### Functions

In [ ]:
# Context manager to suppress logging
@contextlib.contextmanager
def suppress_logging():
    logging.disable(logging.CRITICAL)
    try:
        yield
    finally:
        logging.disable(logging.NOTSET)

# Function to load and preprocess data
def load_and_preprocess(train_file_path, test_file_path):
    train_data = pd.read_csv(train_file_path)
    test_data = pd.read_csv(test_file_path)

    train_data['as_of_date_id'] = train_data['as_of_date_id'].astype(int)
    train_data['age_bin_id'] = train_data['age_bin_id'].astype(str)
    train_data['gender_id'] = train_data['gender_id'].astype(str)
    train_data['district_id'] = train_data['district_id'].astype(str)

    test_data['as_of_date_id'] = test_data['as_of_date_id'].astype(int)
    test_data['age_bin_id'] = test_data['age_bin_id'].astype(str)
    test_data['gender_id'] = test_data['gender_id'].astype(str)
    test_data['district_id'] = test_data['district_id'].astype(str)

    # Smoothing time period 76
    for age_bin in train_data['age_bin_id'].unique():
        for gender in train_data['gender_id'].unique():
            for district in train_data['district_id'].unique():
                mask = (train_data['age_bin_id'] == age_bin) & (train_data['gender_id'] == gender) & (train_data['district_id'] == district)
                count_75 = train_data.loc[mask & (train_data['as_of_date_id'] == 75), 'count'].values
                count_77 = train_data.loc[mask & (train_data['as_of_date_id'] == 77), 'count'].values
                if len(count_75) > 0 and len(count_77) > 0:
                    avg_count = (count_75[0] + count_77[0]) / 2
                    train_data.loc[mask & (train_data['as_of_date_id'] == 76), 'count'] = avg_count

    # Filter train_data to start from as_of_date_id 70
    train_data = train_data[train_data['as_of_date_id'] >= 70].reset_index(drop=True)

    # Assume start date and convert 'as_of_date_id' to datetime
    start_date = pd.to_datetime('2000-01-01')
    train_data['ds'] = start_date + pd.to_timedelta(train_data['as_of_date_id'], unit='D')
    test_data['ds'] = start_date + pd.to_timedelta(test_data['as_of_date_id'], unit='D')

    return train_data, test_data

# Function to normalize data
def normalize_data(df, column):
    mean = df[column].mean()
    std = df[column].std()
    df[column] = (df[column] - mean) / std
    return mean, std

# Function to denormalize data
def denormalize_data(df, column, mean, std):
    df[column] = df[column] * std + mean
    return df

# Function to train models using Prophet
def train_models(train_data, specific_hyperparams=None):
    models = {}
    unique_combinations = train_data[['district_id', 'age_bin_id', 'gender_id']].drop_duplicates()

    for _, row in unique_combinations.iterrows():
        district, age_bin, gender = row['district_id'], row['age_bin_id'], row['gender_id']
        mask = (train_data['district_id'] == district) & (train_data['age_bin_id'] == age_bin) & (train_data['gender_id'] == gender)
        subset_data = train_data[mask]

        # Normalize the data
        mean, std = normalize_data(subset_data, 'count')
        
        subset_data = subset_data.rename(columns={'ds': 'ds', 'count': 'y'})

        # Use specific hyperparameters if provided for the specific case
        if specific_hyperparams and (district, age_bin, gender) in specific_hyperparams:
            hyperparams = specific_hyperparams[(district, age_bin, gender)]
            model = Prophet(
                yearly_seasonality=hyperparams.get('yearly_seasonality', 'auto'),
                changepoint_prior_scale=hyperparams.get('changepoint_prior_scale', 0.4),
                seasonality_prior_scale=hyperparams.get('seasonality_prior_scale', 10),
                changepoint_range=hyperparams.get('changepoint_range', 0.7)
            )
            model.add_seasonality(
                name='12-period',
                period=12,
                fourier_order=hyperparams.get('fourier_order', 10)
            )
        else:
            model = Prophet(
                yearly_seasonality='auto',  # Enable yearly seasonality
                changepoint_prior_scale=0.4,  # Control trend flexibility
                seasonality_prior_scale=10,  # Control seasonality flexibility
                changepoint_range=0.7  # Allow more flexibility for trend changes
            )
            model.add_seasonality(
                name='12-period',
                period=12,
                fourier_order=10 # Seasonal sharpness
            )

        with suppress_logging():
            model.fit(subset_data[['ds', 'y']])
        
        models[(district, age_bin, gender)] = (model, mean, std)
    return models

# Function to make predictions
def make_predictions(models, test_data):
    predictions = []

    unique_combinations = test_data[['district_id', 'age_bin_id', 'gender_id']].drop_duplicates()

    for _, row in unique_combinations.iterrows():
        district, age_bin, gender = row['district_id'], row['age_bin_id'], row['gender_id']
        model, mean, std = models[(district, age_bin, gender)]
        mask_test = (test_data['district_id'] == district) & (test_data['age_bin_id'] == age_bin) & (test_data['gender_id'] == gender)
        subset_test_data = test_data[mask_test]

        future = subset_test_data[['ds']]
        forecast = model.predict(future)

        # Denormalize the predictions
        forecast['yhat'] = forecast['yhat'] * std + mean

        with warnings.catch_warnings():
            warnings.simplefilter('ignore', category=pd.errors.SettingWithCopyWarning)
            subset_test_data['Prediction'] = forecast['yhat'].values
            subset_test_data['Prediction'] = subset_test_data['Prediction'].iloc[::-1].values

        predictions.append(subset_test_data[['ID', 'district_id', 'age_bin_id', 'gender_id', 'as_of_date_id', 'Prediction']])
    
    return pd.concat(predictions, ignore_index=True)

# Function to plot results
def plot_results(train_data, predictions):
    train_plot_df = train_data.groupby('as_of_date_id')['count'].sum().reset_index()
    fig_train = px.line(
        train_plot_df, x='as_of_date_id', y='count',
        title='Sum of Population per Period of Time (Training Data)',
        labels={'as_of_date_id': 'Time Period', 'count': 'Total Population'},
        markers=True)
    
    test_plot_df = predictions.groupby('as_of_date_id')['Prediction'].sum().reset_index()
    fig_test = px.line(
        test_plot_df, x='as_of_date_id', y='Prediction',
        title='Sum of Population per Period of Time (Test Predictions)',
        labels={'as_of_date_id': 'Time Period', 'Prediction': 'Total Predicted Population'},
        markers=True)
    
    # Change the color of the prediction line
    fig_test.data[0].line.color = 'red'
    
    fig_train.add_trace(fig_test.data[0])
    fig_train.update_layout(
        title='Population Count per Period of Time (Training Data and Test Predictions)',
        xaxis_title='Time Period',
        yaxis_title='Population Count',
        legend_title='Dataset')
    
    # Calculate the min and max values for y-axis
    all_y_values = pd.concat([train_plot_df['count'], test_plot_df['Prediction']])
    y_min = all_y_values.min() * 0.99
    y_max = all_y_values.max() * 1.01

    # Add vertical lines for specified time periods
    vertical_lines = [82, 94, 106, 118]
    for period in vertical_lines:
        fig_train.add_shape(
            type="line",
            x0=period,
            y0=y_min,
            x1=period,
            y1=y_max,
            line=dict(color="Green", width=2, dash="dash")
        )
    
    fig_train.show()


# Function to plot results for a specific combination
def plot_individual_results(district, age_bin, gender):
    train_subset = train_data[(train_data['district_id'] == district) & 
                              (train_data['age_bin_id'] == age_bin) & 
                              (train_data['gender_id'] == gender)]
    
    pred_subset = predictions[(predictions['district_id'] == district) & 
                              (predictions['age_bin_id'] == age_bin) & 
                              (predictions['gender_id'] == gender)]
    
    # Fit a linear regression to the training data
    X_train = train_subset['as_of_date_id'].values.reshape(-1, 1)
    y_train = train_subset['count'].values
    linear_regressor = LinearRegression()
    linear_regressor.fit(X_train, y_train)
    
    # Create a range for the trend line from the start of training to the end of predictions
    start_train = train_subset['as_of_date_id'].min()
    end_pred = pred_subset['as_of_date_id'].max()
    X_trend = np.arange(start_train, end_pred + 1).reshape(-1, 1)
    y_trend = linear_regressor.predict(X_trend)
    
    # Create a dataframe for the trend line
    trend_df = pd.DataFrame({
        'as_of_date_id': X_trend.flatten(),
        'trend': y_trend
    })

    fig = go.Figure()
    
    # Add training data
    fig.add_trace(go.Scatter(
        x=train_subset['as_of_date_id'], 
        y=train_subset['count'], 
        mode='lines+markers', 
        name='Training Data',
        line=dict(color='blue')
    ))
    
    # Add prediction data
    fig.add_trace(go.Scatter(
        x=pred_subset['as_of_date_id'], 
        y=pred_subset['Prediction'], 
        mode='lines+markers', 
        name='Predictions',
        line=dict(color='red')
    ))

    # Add trend line
    fig.add_trace(go.Scatter(
        x=trend_df['as_of_date_id'], 
        y=trend_df['trend'], 
        mode='lines', 
        name='Trend Line',
        line=dict(color='rgba(0, 0, 0, 0.5)', width=1, dash="dash"),
        showlegend=True
    ))

    # Calculate the min and max values for y-axis
    all_y_values = pd.concat([train_subset['count'], pred_subset['Prediction'], trend_df['trend']])
    y_min = all_y_values.min() * 0.99
    y_max = all_y_values.max() * 1.01

    # Add vertical lines for specified time periods
    vertical_lines = [82, 94, 106, 118]
    for period in vertical_lines:
        fig.add_shape(
            type="line",
            x0=period,
            y0=y_min,
            x1=period,
            y1=y_max,
            line=dict(color="Green", width=2, dash="dash")
        )
    
    fig.update_layout(
        title=f'Population Count per Period of Time (District: {district}, Age Bin: {age_bin}, Gender: {gender})',
        xaxis_title='Time Period',
        yaxis_title='Population Count',
        legend_title='Dataset',
        height=500
    )
    
    return fig

### Uploading data/training model/predicting/plotting general overview

In [ ]:
# Main execution
train_file_path = 'data/train.csv'
test_file_path = 'data/test.csv'

# Load and preprocess data
train_data, test_data = load_and_preprocess(train_file_path, test_file_path)

# Specify sets that needed to be changed from general model
specific_hyperparams = {
    ('0', '1', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.6, 'seasonality_prior_scale': 20, 'changepoint_range': 0.7, 'fourier_order': 10},
    ('0', '4', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.3, 'seasonality_prior_scale': 5, 'changepoint_range': 0.9, 'fourier_order': 10},
    ('0', '5', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 15, 'changepoint_range': 0.5, 'fourier_order': 15},
    ('0', '5', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 10, 'changepoint_range': 0.5, 'fourier_order': 10},

    ('1', '1', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 15, 'changepoint_range': 0.95, 'fourier_order': 5},
    ('1', '4', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.3, 'seasonality_prior_scale': 10, 'changepoint_range': 0.95, 'fourier_order': 7},
    ('1', '4', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.7, 'seasonality_prior_scale': 5, 'changepoint_range': 0.9, 'fourier_order': 10},

    ('2', '2', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 1, 'changepoint_range': 0.8, 'fourier_order': 1},
    ('2', '2', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.3, 'seasonality_prior_scale': 5, 'changepoint_range': 0.95, 'fourier_order': 1},
    ('2', '5', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.7, 'seasonality_prior_scale': 10, 'changepoint_range': 0.95, 'fourier_order': 20},

    ('3', '1', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 5, 'changepoint_range': 0.5, 'fourier_order': 7},

    ('5', '4', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 1.0, 'seasonality_prior_scale': 15, 'changepoint_range': 0.95, 'fourier_order': 15},

    ('6', '4', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.3, 'seasonality_prior_scale': 1, 'changepoint_range': 0.95, 'fourier_order': 7},

    ('7', '4', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 5, 'changepoint_range': 0.95, 'fourier_order': 10},

    ('8', '4', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.4, 'seasonality_prior_scale': 1, 'changepoint_range': 0.95, 'fourier_order': 5},
    ('8', '4', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 20, 'changepoint_range': 0.8, 'fourier_order': 5},

    ('9', '3', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 5, 'changepoint_range': 0.9, 'fourier_order': 5},

    ('12', '4', '0'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.7, 'seasonality_prior_scale': 5, 'changepoint_range': 0.5, 'fourier_order': 5},

    ('15', '4', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.3, 'seasonality_prior_scale': 15, 'changepoint_range': 0.7, 'fourier_order': 1},

    ('16', '3', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 1.0, 'seasonality_prior_scale': 1, 'changepoint_range': 0.95, 'fourier_order': 1},

    ('17', '4', '1'): {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.4, 'seasonality_prior_scale': 1, 'changepoint_range': 0.9, 'fourier_order': 1},
}

# Train models for each combination of district, age_bin, and gender using Prophet
models = train_models(train_data, specific_hyperparams)

# Make predictions using the trained models
predictions = make_predictions(models, test_data)
# Adjusted prediction level, fixing model overfitting
predictions['Prediction'] = predictions['Prediction'] * 1.0035

# Plot combined results
plot_results(train_data, predictions)

### Plotting overview case by case

In [ ]:
# Create a dropdown for each combination to view individual plots
unique_combinations = train_data[['district_id', 'age_bin_id', 'gender_id']].drop_duplicates()

# Convert district_id to integers for correct sorting, then sort by district_id, age_bin_id, and gender_id
unique_combinations['district_id_int'] = unique_combinations['district_id'].astype(int)
unique_combinations = unique_combinations.sort_values(by=['district_id_int', 'age_bin_id', 'gender_id'])

# Create the options for the dropdown
options = [(f'District: {row["district_id"]}, Age Bin: {row["age_bin_id"]}, Gender: {row["gender_id"]}', 
            (row["district_id"], row["age_bin_id"], row["gender_id"])) for _, row in unique_combinations.iterrows()]

# Drop the temporary integer column
unique_combinations = unique_combinations.drop(columns=['district_id_int'])


# Initialize the figure widget
fig = go.FigureWidget()

def update_plot(change):
    district, age_bin, gender = change['new']
    new_fig = plot_individual_results(district, age_bin, gender)
    
    with fig.batch_update():
        fig.data = []
        for trace in new_fig.data:
            fig.add_trace(trace)
        fig.layout = new_fig.layout

# Create the dropdown widget
dropdown = Dropdown(options=options, description='Select Group:')
dropdown.observe(update_plot, names='value')

# Create 'Back' and 'Next' buttons
back_button = Button(description='Back')
next_button = Button(description='Next')

# Function to handle 'Back' button click
def on_back_button_clicked(b):
    current_index = [i[1] for i in options].index(dropdown.value)
    if current_index > 0:
        dropdown.value = options[current_index - 1][1]

# Function to handle 'Next' button click
def on_next_button_clicked(b):
    current_index = [i[1] for i in options].index(dropdown.value)
    if current_index < len(options) - 1:
        dropdown.value = options[current_index + 1][1]

back_button.on_click(on_back_button_clicked)
next_button.on_click(on_next_button_clicked)

# Display the dropdown, buttons, and the plot output
display(VBox([HBox([back_button, next_button, dropdown]), fig]))

# Initialize with the first combination
dropdown.value = options[0][1]
update_plot({'new': options[0][1]})

### Saving output

In [ ]:
# Save predictions to 'output.csv'
output = predictions[['ID', 'Prediction']].rename(columns={'Prediction': 'count'}).sort_values(by='ID')
output.to_csv('output.csv', index=False)